# Importing the necessary Libraries

In [2]:
import os, json, joblib, numpy as np, pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

from keras.utils import Sequence, to_categorical, pad_sequences
from keras.models import Model, load_model
from keras.layers import (
    Input, Conv1D, BatchNormalization, Activation, add, MaxPooling1D, Dropout,
    Bidirectional, LSTM, GlobalAveragePooling1D, Dense, Multiply, Reshape,
    Lambda, Concatenate, GRU, GaussianNoise
)
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import backend as KerasTensor
import tensorflow as tf
import polars as pl
from sklearn.model_selection import StratifiedGroupKFold
from scipy.spatial.transform import Rotation as R

# Fix Seed

In [3]:
import random
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
seed_everything(seed=42)

# Configuration

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices("GPU"))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

# (任意) 初回のメモリ全確保を防ぐ
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

print("Devices:", tf.config.list_physical_devices())
print("Logical GPUs:", tf.config.list_logical_devices('GPU'))

# ちょい重め計算でGPUを使うか確認
with tf.device('/GPU:0'):
    a = tf.random.normal([4000, 4000])
    b = tf.random.normal([4000, 4000])
    c = tf.matmul(a, b)
    print("matmul ok, shape:", c.shape)

# 簡単な学習（MNIST）
(x_train, y_train), _ = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32")/255.0
x_train = x_train[..., None]

model = keras.Sequential([
    layers.Conv2D(32, 3, activation="relu", input_shape=(28,28,1)),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax"),
])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=512, epochs=1)  # 1エポックだけ動けば十分


Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Logical GPUs: [LogicalDevice(name='/device:GPU:0', device_type='GPU')]
matmul ok, shape: (4000, 4000)


I0000 00:00:1755379773.316144  145045 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5561 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
2025-08-17 06:29:35.773841: I external/local_xla/xla/service/service.cc:163] XLA service 0x75e858005920 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-17 06:29:35.773900: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-08-17 06:29:35.813438: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-17 06:29:35.977580: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91200
2025-08-17 06:29:36.591783: I external/local_xla/xla/stream_executor/cuda/subprocess_compilati

 22/118 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5428 - loss: 1.5918

I0000 00:00:1755379778.824861  145288 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


116/118 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7791 - loss: 0.7923

2025-08-17 06:29:40.274378: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_375', 4 bytes spill stores, 4 bytes spill loads



118/118 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.8841 - loss: 0.4177
